In [1]:
from supabase.client import Client
from typing import Dict, Any, Tuple, List
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

In [2]:
from utils.utils import initialize_environment_variables, initialize_subabase_client, initialize_openai_client
from utils.utils import initialize_tavily_client
from api.config import interface_config

initialize_environment_variables("../.env")

# Initialize Supabase Client
supabase_client = initialize_subabase_client()

# Initialize OpenAI Client
openai_client = initialize_openai_client()

# Initialize Tavily Client
tavily_client = initialize_tavily_client()

In [3]:
input_dict = {
"school_type": "Gymnasium",
"subject": "English",
"topic": "Shakespeare",
"grade": "13.",
"state": "Bayern",
"keywords": ["shakespeare", "poetry", "analysis"], 
"context": ['docs/Pedagogy.pdf']
}

In [4]:
input_dict

{'school_type': 'Gymnasium',
 'subject': 'English',
 'topic': 'Shakespeare',
 'grade': '13.',
 'state': 'Bayern',
 'keywords': ['shakespeare', 'poetry', 'analysis'],
 'context': ['uuid1', 'uuid2']}

In [5]:
def unpack_prompt_input(input: Dict[str, Any]) -> Tuple[str, str, str, str, List[str], List[str]]:
    school_type = str(input.get("subject"))
    subject = str(input.get("subject"))
    topic = str(input.get("topic"))
    grade = str(input.get("grade"))
    state = str(input.get("state"))
    keywords = [str(i) for i in input.get("keywords")]
    context = [str(i) for i in input.get("context")]
    return school_type, subject, topic, grade, state, keywords, context

In [6]:
school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(input_dict)

In [7]:
def create_task_string(school_type, subject, topic, grade, state, keywords, context) -> str:
    keywords = '\n'.join(keywords)
    return f"""Plane eine Unterrichtsstunde im Fach {subject} zum Thema {topic} für eine {grade} Klasse in {state} für den Schultyp {school_type}.
Lege bei dieser Aufgabe einen besonderen Fokus auf die folgenden Schlüsselbegriffe:
    
Schlüsselbegriffe:
------------------
{keywords}
------------------"""

In [8]:
task = create_task_string(school_type, subject, topic, grade, state, keywords, context)
print(task)

Plane eine Unterrichtsstunde im Fach English zum Thema Shakespeare für eine 13. Klasse in Bayern für den Schultyp English.
Lege bei dieser Aufgabe einen besonderen Fokus auf die folgenden Schlüsselbegriffe:
    
Schlüsselbegriffe:
------------------
shakespeare
poetry
analysis
------------------


In [9]:
def get_uploaded_ids(filepath, upload_table_name):
    data, _cnt = supabase_client.table(upload_table_name).select('id').contains('metadata', {'source':filepath}).execute()
    return [data[1][i].get("id") for i in range(len(data[1]))]

In [10]:
def create_filter_list(context):
    uploaded_ids = []
    for doc in context:
        if len(doc) > 0:
            doc_uploaded_ids = get_uploaded_ids(doc, interface_config.upload_table_name)
            if len(doc_uploaded_ids) > 0:
                uploaded_ids += doc_uploaded_ids
    return uploaded_ids

In [11]:
filter_list = create_filter_list(context)

2024-02-02 12:59:24,184:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/vector_store?select=id&metadata=cs.%7B%22source%22%3A%20%22uuid1%22%7D "HTTP/1.1 200 OK"
2024-02-02 12:59:24,278:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/vector_store?select=id&metadata=cs.%7B%22source%22%3A%20%22uuid2%22%7D "HTTP/1.1 200 OK"


In [12]:
def initialize_vector_store(client: Client, embedding: OpenAIEmbeddings, table_name: str, query_name: str):
    vector_store = SupabaseVectorStore(
        client=client,
        embedding=embedding,
        table_name=table_name,
        query_name=query_name,
    )
    return vector_store

In [13]:
embedding = OpenAIEmbeddings()

In [14]:
vector_store = initialize_vector_store(supabase_client, embedding, interface_config.upload_table_name, interface_config.supabase_match_function) 

In [15]:
retrieved_context = vector_store.similarity_search(task,k=2,filter=filter_list)

2024-02-02 12:59:29,067:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-02 12:59:29,169:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=2 "HTTP/1.1 200 OK"


In [16]:
print(type(retrieved_context[0]))

IndexError: list index out of range

In [17]:
def create_prompt_template(retrieved_context, task) -> str:
    return f"""Du bist ein Assistent für Lehrende und hilfst ihnen dabei ihren Unterricht vorzubereiten. 
Verwende dazu die Informationen im Kontext, um die Aufgabe am Ende zu lösen. Gib den Schlüsselwötern bei der Lösung der Aufgabe ein besonders hohes Gewicht.
Wenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden. 
Die Lösung der Aufgabe soll eine inhaltliche und zeitliche Struktur beinhalten.

Kontext:
---------
{retrieved_context}
---------

Los geht's!

Aufgabe: {task}"""
prompt_template = create_prompt_template(retrieved_context, task)

In [18]:
print(prompt_template)

Du bist ein Assistent für Lehrende und hilfst ihnen dabei ihren Unterricht vorzubereiten. 
Verwende dazu die Informationen im Kontext, um die Aufgabe am Ende zu lösen. Gib den Schlüsselwötern bei der Lösung der Aufgabe ein besonders hohes Gewicht.
Wenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden. 
Die Lösung der Aufgabe soll eine inhaltliche und zeitliche Struktur beinhalten.

Kontext:
---------
[]
---------

Los geht's!

Aufgabe: Plane eine Unterrichtsstunde im Fach English zum Thema Shakespeare für eine 13. Klasse in Bayern für den Schultyp English.
Lege bei dieser Aufgabe einen besonderen Fokus auf die folgenden Schlüsselbegriffe:
    
Schlüsselbegriffe:
------------------
shakespeare
poetry
analysis
------------------


In [20]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [21]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

In [22]:
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="vector_store",
        query_name="match_documents_with_uuid_filter",
    )
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 4, 'lambda_mult': 0.25, 'filter': filter_list})
retriever_tool = tool = create_retriever_tool(
    retriever,
    "get_more_context",
    "Retrieve more relevant context information.",
)

In [23]:
tools = [retriever_tool, tavily_tool]

In [24]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
agent = create_openai_functions_agent(llm, tools, prompt)

In [30]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [31]:
response = agent_executor.invoke({"input": prompt_template})



> Entering new AgentExecutor chain...


2024-01-30 21:15:50,077:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `tavily_search_results_json` with `{'query': 'Unterrichtsstunde zum Thema Shakespeare für eine 13. Klasse in Bayern'}`


[{'url': 'https://schulportal.de/?cmd=suche&s=Klausur+Shakespeare&smd[f][3]=3&smd[ks][1100]=1100&smd[ks][12]=12&smd[ks][11]=11&ls=80', 'content': 'Shakespeare, Hamlet (ehem LK) Schulaufgabe Englisch 12 Bayern  Unterrichtsmaterial ´Klausur Shakespeare´, Englisch, Klasse 13 LK+13 GK+12+11  Lehrprobe LK12 Shakespeare- a literary ´giant´ in the 21st century (ehem LK) William Shakespeare Lehrplan Englisch  Sachtext zur Autorenschaft Shakespeare´s Klassenarbeit Englisch 12 Nordrh.-Westf.106 Dokumente Suche ´Klausur Shakespeare´, Englisch, Klasse 13 LK+13 GK+12+11. die größte Plattform für kostenloses Unterrichtsmaterial. ... Englisch Kl. 12, Gymnasium/FOS, Bayern 39 KB. Shakespeare, Hamlet (ehem LK) ... Eine Unterrichtsstunde mit dem Ziel, neue Einsichten in die Charaktere Juliets und Capulets (Akt3/Szene5) durch die ...'}, {'url': 'https://schulportal.de/?cmd=s

2024-01-30 21:15:58,434:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hier ist ein Vorschlag für eine Unterrichtsstunde zum Thema Shakespeare für eine 13. Klasse im Gymnasium. Die Schlüsselbegriffe sind Shakespeare, Poesie und Analyse.

**Thema:** Einführung in Shakespeares Poesie und Analyse seiner Werke

**Dauer:** 60 Minuten

**Materialien:** Kopien von Shakespeares Gedichten, Whiteboard, Marker, Laptop und Beamer für Präsentationen

**Unterrichtsablauf:**

1. **Einführung (10 Minuten):** Beginnen Sie mit einer kurzen Einführung in das Leben und Werk von William Shakespeare. Erklären Sie den Schülern, warum seine Werke auch heute noch relevant sind. 

2. **Einführung in Shakespeares Poesie (15 Minuten):** Präsentieren Sie einige von Shakespeares bekanntesten Gedichten und Sonetten. Diskutieren Sie die Themen, die in seiner Poesie häufig auftauchen, wie Liebe, Tod, Zeit und Schönheit.

3. **Analyse von Shakespeares Poesie (20 Minuten):** Teilen Sie die Klasse in Gruppen auf und geben Sie jeder Gruppe ein Gedicht oder Sonett von Shakespeare zur Analyse.

In [34]:
response["context"] = retrieved_context

In [35]:
response

{'input': "Du bist ein Assistent für Lehrende und hilfst ihnen dabei ihren Unterricht vorzubereiten. \nVerwende dazu die Informationen im Kontext, um die Aufgabe am Ende zu lösen. Gib den Schlüsselwötern bei der Lösung der Aufgabe ein besonders hohes Gewicht.\nWenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden. \nDie Lösung der Aufgabe soll eine Struktur der angefragten Unterrichtsstunde enthalten inklusive einer zeitlichen Struktur.\n\nKontext:\n---------\n[Document(page_content='Lehrplanauszüge\\n4 30.01.2024(3) Beim Kompetenzerwerb zeigen sich Schülerinnen und Schüler des  \\nGymnasiums geistig besonders beweglich und fantasievoll, sie lernen  \\nschnell, gern und zielstrebig und verfügen über ein gutes Gedächtnis. Sie  \\nsind bereit, sich ausdauernd und unter verschiedenen Blickwinkeln mit  \\nDenk- und Gestaltungsaufgaben auseinanderzusetzen, und entwickeln dabei  \\nzunehmend die Fähigkeit zur Abstraktion, z

In [14]:
def download_file_from_bucket(destination: str, full_path: str, path: str):
    full_path_list = full_path.split("/")
    bucket_name = full_path_list[0]
    with open(destination, 'wb+') as f:
        storage_file = supabase_client.storage.from_(bucket_name).download(path)
        f.write(storage_file)

In [15]:
download_file_from_bucket("RMH-Rechnung_12_23.pdf", "context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf", "user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf")

2024-02-01 18:23:05,084:INFO - HTTP Request: GET https://smxwnqdpxcvclxhbvhsf.supabase.co/storage/v1/object/context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf "HTTP/1.1 200 OK"


In [1]:
upload_dict = {"path":"user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf","id":"c2f81078-039d-435a-b0c6-84534ac9048f","fullPath":"context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf"}

In [2]:
full_path = upload_dict['fullPath']

In [3]:
full_path

'context_uploads/user_2e4ace06-270b-429b-9726-f937261ed293/RMH-Rechnung_12_23.pdf'

In [6]:
full_path.split("/")

'context_uploads'